# Cross-validation

A **Cross-validation**, às vezes chamada de **rotation estimation**, é qualquer uma das várias técnicas de validação de modelo semelhantes para avaliar como os resultados de uma análise estatística serão generalizados para um conjunto de dados independente.

É usado principalmente em configurações onde o objetivo é a previsão e se deseja estimar a precisão com que um modelo preditivo será executado na prática.

Em um problema de predição, um modelo geralmente recebe um conjunto de dados conhecidos no qual o treinamento é executado (conjunto de dados de treinamento) e um conjunto de dados desconhecidos (ou dados vistos pela primeira vez) contra o qual o modelo é testado (chamado de conjunto de dados de validação ou teste).

![img](https://i.ibb.co/4mBSH9P/K-fold-cross-validation-EN.png)

O objetivo da Cross-validation é testar a capacidade do modelo de prever novos dados que não foram usados na estimativa, a fim de sinalizar problemas como **overfitting** ou **selection bias** e dar uma visão de como o modelo irá generalizar para um conjunto de dados independente (ou seja, um conjunto de dados desconhecido, por exemplo, de um problema real).

Para mais detalhes você pode checar o **[Wikipedia](https://en.wikipedia.org/wiki/Cross-validation_(statistics))**

Vamos agora executar experimentos para melhor compreendermos o conceito de Cross-Validation.

## Dados

Primeiramente vamos lidar com um problema de regressão envolvendo dados relacionados a propaganda e seu devido resultado em vendas para uma determinada corporação.

Inicialmente importamos as bibliotecas necessárias para trabalharmos.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

E agora carregamos os nossos dados em um **DataFrame** pandas.

In [18]:
df = pd.read_csv('dados/Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


## Procedimento de Divisão: Treinamento | Teste

0. Limpar e ajustar os dados conforme necessário para **X** e **y**
1. Dividir os dados em Treinamento/Teste para ambos **X** e **y**
2. Ajustar/Treinar um Scaler nos dados de treinamento **X**
3. Fazer o Scale dos dados de teste **X**
4. Criar o Modelo
5. Ajustar/Treinar o Modelo nos dados de treinamento **X**
6. Avaliar o Modelo nos dados de treinamento **X** (Ao criar previsões e compará-las com **y_test**)
7. Ajustar os parâmetros conforme necessário e repetir os procedimento **5** e **6**

Vamos então executar essas etapas para o nosso problema atual.

#### 0

In [19]:
X = df.drop('sales', axis=1)
y = df['sales']

#### 1

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#### 2-3

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#### 4-5

In [22]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100)
model.fit(X_train, y_train);

#### 6

In [23]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

9.739601506686308

#### 7

In [24]:
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
y_preds = ridge.predict(X_test)

mean_squared_error(y_test, y_preds)

2.461088644854306

## Divisão: Treinamento | Validação | Teste

Para uma melhor avaliação de nosso modelo, vamos dividir os nossos dados em três conjuntos: **treinamento**, **validação** e **teste**.

In [38]:
X = df.drop('sales', axis=1)
y = df['sales']

Faremos a primeira divisão.

In [39]:
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3)

E a segunda divisão.

In [40]:
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.5)

Para confirmar a exatidão de nossas divisões, podemos verificar o tamanho de cada conjunto.

In [41]:
len(df)

200

In [42]:
len(X_train) + len(X_val) + len(X_test)

200

E agora faremos o scaling dos nossos dados.

In [43]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Criamos um novo modelo.

In [44]:
model_1 = Ridge(alpha=100)
model_1.fit(X_train, y_train)
y_val_pred = model_1.predict(X_val)

Checamos as métricas.

In [45]:
mean_squared_error(y_val, y_val_pred)

5.879842460946637

Mais uma vez, criamos um novo modelo.

In [46]:
model_2 = Ridge(alpha=1)
model_2.fit(X_train, y_train)
new_pred_val = model_2.predict(X_val)

Novamente checamos as métricas.

In [47]:
mean_squared_error(y_val, new_pred_val)

2.450795644643107

Supondo que estamos satisfeitos com este último modelo que desempenhou melhor, podemos então finalmente executar previsões nos dados de teste e avaliar o seu desempenho.

In [48]:
y_final_test_pred = model_2.predict(X_test)

In [49]:
mean_squared_error(y_test, y_final_test_pred)

2.1221630223459815

## Cross-validation

Faremos Cross-Validation em nossos dados com a função **cross_val_score**.

Vamos novamente repetir as etapas tradicionais.

In [50]:
X = df.drop('sales', axis=1)
y = df['sales']

Dessa vez trabalharemos apenas com conjunto de treinamento e teste.

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Fazemos o scaling dos dados.

In [52]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Criamos um modelo para cross-validar a escolha dos parâmetros.

In [53]:
model = Ridge(alpha=100)

In [54]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [57]:
abs(scores.mean())

8.372820360401166

Não obtivemos um bom desempenho neste modelo, podemos então criar outro e avaliá-lo.

In [73]:
model = Ridge(alpha=1)

scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
abs(scores.mean())

3.2938656880340473

Supondo que estamos satisfeitos com esse modelo.

Neste caso, devemos então treiná-lo novamente e finalmente executar previsões nos dados de teste.

In [74]:
model.fit(X_train, y_train)
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test, y_final_test_pred)

2.28650154353358

E este será o nosso **mean_squared_error** final.

## A Função cross_validate

A função **cross_validate()** nos permite ver múltiplas métricas de perfomance da **cross-validation** em um modelo e explorar quanto tempo o ajuste dos dados e o teste levou.

A seguir veremos um exemplo de como esta função opera.

In [75]:
X = df.drop('sales', axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Com nossos dados preparados, podemos testar a função **cross_validate**.

In [76]:
from sklearn.model_selection import cross_validate

model = Ridge(alpha=100)

scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error','neg_mean_absolute_error'], cv=10)

Nosso modelo foi executado em 10 **folds** diferentes e calculou ambas as métricas de desempenho que definimos no parâmetro **scoring**.

Para melhor visualizar esses resultados, podemos construir um **DataFrame** pandas.

In [77]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.002835,0.016784,-7.619863,-2.117068
1,0.002378,0.004966,-4.539462,-1.533362
2,0.002399,0.001550,-9.553552,-2.506458
3,0.001906,0.001924,-8.755653,-2.236681
4,0.002300,0.001730,-4.160177,-1.831017
5,0.002524,0.001527,-8.516092,-2.348971
6,0.001680,0.001152,-3.742907,-1.563091
7,0.001604,0.001114,-10.293452,-2.612610
8,0.001608,0.003474,-2.265035,-1.041094
9,0.008229,0.003213,-13.212076,-3.214754


Para ver a média das pontuações podemos simplesmente chamar a função **mean()**.

In [78]:
scores_df.mean()

fit_time                        0.002746
score_time                      0.003744
test_neg_mean_squared_error    -7.265827
test_neg_mean_absolute_error   -2.100511
dtype: float64

E agora podemos tentar construir um modelo melhor.

In [79]:
model = Ridge(alpha=1)

scores = cross_validate(model, X_train, y_train, scoring=['neg_mean_squared_error','neg_mean_absolute_error'], cv=10)
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.015302,0.003647,-2.793109,-1.301176
1,0.002755,0.002129,-1.794693,-1.103833
2,0.002494,0.001546,-1.808498,-1.142403
3,0.002088,0.001564,-2.706553,-1.356509
4,0.001823,0.001399,-2.236854,-1.325531
5,0.001797,0.001355,-1.255678,-0.844707
6,0.001755,0.001299,-2.959246,-1.290730
7,0.001800,0.001289,-6.018774,-1.753217
8,0.001639,0.001157,-2.099935,-1.158073
9,0.001653,0.001283,-4.187740,-1.846565


In [80]:
scores_df.mean()

fit_time                        0.003311
score_time                      0.001667
test_neg_mean_squared_error    -2.786108
test_neg_mean_absolute_error   -1.312274
dtype: float64

De fato conseguimos obter um modelo mais eficaz.

Agora podemos realizar previsões com este modelo e testá-lo em uma métrica final.

In [82]:
model.fit(X_train, y_train)
y_final_pred = model.predict(X_test)
mean_squared_error(y_test, y_final_pred)

3.5465657634631875

## Grid Search

Muitas vezes modelos complexos possuem múltiplos hiperparâmetros ajustáveis.

Grid Search é uma maneira de treinar e validar o modelo em todas as possíveis combinações de múltiplas opções de hiperparâmetros.

Scikit-Learn inclui uma classe **GridSearchCV** capaz de testar um dicionário de múltiplas opções de hiperparâmetros através de **Cross-validation**.

Isso permite que ambos cross-validation e grid search sejam executamos de forma generalizada para qualquer modelo.

Vamos então explorar essa técnica, começamos executando as etapas tradicionais.

In [83]:
X = df.drop('sales', axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Então iremos criar um modelo padrão base e definir uma grid de hiperparâmetros.

In [85]:
from sklearn.linear_model import ElasticNet

base_elastic_net_model = ElasticNet()

In [86]:
param_grid = {'alpha':[0.1, 1, 5, 10, 50, 100], 'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]}

E então executamos o **Grid Search**.

In [88]:
from sklearn.model_selection import GridSearchCV

grid_model = GridSearchCV(estimator=base_elastic_net_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    0.8s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=1)

E agora vejamos qual é o melhor estimador.

In [89]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=1,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

Também podemos ver quais os melhores hiperparâmetros encontrados.

In [90]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

Também podemos obter todas as informações de todas as combinações do Grid Search.

In [93]:
pd.DataFrame(grid_model.cv_results_).T

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
mean_fit_time,0.00468302,0.00248928,0.0020524,0.00279469,0.00395298,0.00371737,0.00351739,0.00452728,0.0069778,0.00598488,...,0.00101357,0.00121608,0.000976896,0.00505219,0.000948048,0.000994444,0.00124798,0.000968933,0.000991869,0.00141497
std_fit_time,0.00238295,0.00058354,0.000624266,0.00141403,0.00095762,0.00138793,0.000923927,0.00238094,0.00305583,0.00269613,...,0.000140988,0.000394847,9.9147e-05,0.00806883,3.46758e-05,7.94504e-05,0.00033511,4.50203e-05,8.92763e-05,0.000607852
mean_score_time,0.00161228,0.00170956,0.000858688,0.000977755,0.00279694,0.00167489,0.00194507,0.0013422,0.00311418,0.00252714,...,0.000444555,0.000726795,0.0004076,0.000521135,0.000439405,0.000415802,0.000752163,0.000410318,0.000482607,0.000550699
std_score_time,0.000662964,0.000696013,0.000177427,0.000291092,0.000979012,0.00104375,0.00094467,0.00064523,0.00128013,0.00169444,...,5.06574e-05,0.000393601,1.95437e-05,0.000216457,5.77979e-05,1.94611e-05,0.000413355,2.03917e-05,0.000134438,0.000284207
param_alpha,0.1,0.1,0.1,0.1,0.1,0.1,1,1,1,1,...,50,50,50,50,100,100,100,100,100,100
param_l1_ratio,0.1,0.5,0.7,0.95,0.99,1,0.1,0.5,0.7,0.95,...,0.7,0.95,0.99,1,0.1,0.5,0.7,0.95,0.99,1
params,"{'alpha': 0.1, 'l1_ratio': 0.1}","{'alpha': 0.1, 'l1_ratio': 0.5}","{'alpha': 0.1, 'l1_ratio': 0.7}","{'alpha': 0.1, 'l1_ratio': 0.95}","{'alpha': 0.1, 'l1_ratio': 0.99}","{'alpha': 0.1, 'l1_ratio': 1}","{'alpha': 1, 'l1_ratio': 0.1}","{'alpha': 1, 'l1_ratio': 0.5}","{'alpha': 1, 'l1_ratio': 0.7}","{'alpha': 1, 'l1_ratio': 0.95}",...,"{'alpha': 50, 'l1_ratio': 0.7}","{'alpha': 50, 'l1_ratio': 0.95}","{'alpha': 50, 'l1_ratio': 0.99}","{'alpha': 50, 'l1_ratio': 1}","{'alpha': 100, 'l1_ratio': 0.1}","{'alpha': 100, 'l1_ratio': 0.5}","{'alpha': 100, 'l1_ratio': 0.7}","{'alpha': 100, 'l1_ratio': 0.95}","{'alpha': 100, 'l1_ratio': 0.99}","{'alpha': 100, 'l1_ratio': 1}"
split0_test_score,-2.8928,-2.72941,-2.64764,-2.55142,-2.53672,-2.53308,-9.46049,-8.38233,-7.53369,-6.06217,...,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355,-29.3355
split1_test_score,-2.48542,-2.50557,-2.52162,-2.5512,-2.55701,-2.55851,-5.35221,-4.74699,-4.31287,-3.61182,...,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893,-19.2893
split2_test_score,-3.41803,-3.21681,-3.12281,-3.02473,-3.0157,-3.01348,-9.2667,-7.81779,-6.81869,-5.5276,...,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008,-25.6008


Finalmente podemos realizar previsões, lembrando que Scikit-Learn automaticamente utilizará os melhores hiperparâmetros.

In [94]:
y_pred = grid_model.predict(X_test)

E agora pode medir o desempenho com a métrica que desejarmos.

In [95]:
mean_squared_error(y_test, y_pred)

3.639585218463167

Temos então o resultado final para a métrica **mean_squared_error**.

Podemos também retomar o processo de **Grid Search** e buscar por mais combinações de hiperparâmetros ou finalizar com este resultado.